In [1]:
%matplotlib inline

In [2]:
import keras

Using TensorFlow backend.


In [3]:
from math import ceil

In [4]:
from datetime import datetime

In [5]:
import json

In [6]:
DATA_PATH = '/data/'

In [7]:
MODEL_PATH = 'models/'

In [8]:
BATCH_SIZE = 128

## Load data

In [9]:
from keras.applications.xception import Xception

In [9]:
from keras.preprocessing import image

In [11]:
from keras.models import Model

In [12]:
from keras.layers import Dense, GlobalAveragePooling2D

In [10]:
from keras.optimizers import Adam

In [11]:
train_generator = image.ImageDataGenerator(
    rotation_range=10, 
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

In [12]:
train_batches = train_generator.flow_from_directory(
    DATA_PATH + 'train/',
    target_size=(299, 299),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

Found 15655 images belonging to 10 classes.


In [13]:
valid_batches = image.ImageDataGenerator().flow_from_directory(
    DATA_PATH + 'valid/',
    target_size=(299, 299),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

Found 1967 images belonging to 10 classes.


In [14]:
num_classes = train_batches.num_classes

In [18]:
base_model = Xception(weights='imagenet', include_top=False)

In [19]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [20]:
model = Model(inputs=base_model.input, outputs=predictions)

In [21]:
for layer in base_model.layers:
    layer.trainable = False

In [20]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [62]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=1,
)

Epoch 1/1


/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:888: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


61/62 [============================>.] - ETA: 4s - loss: 2.1079 - acc: 0.2494

/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (97751910 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


62/62 [==============================] - 361s 6s/step - loss: 2.1057 - acc: 0.2511 - val_loss: 2.0277 - val_acc: 0.3086


In [63]:
model.optimizer.lr = 0.01

In [64]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=10,
)

Epoch 1/10
 3/62 [>.............................] - ETA: 4:44 - loss: 1.9792 - acc: 0.3203

/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:888: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


61/62 [============================>.] - ETA: 4s - loss: 1.9756 - acc: 0.3056

/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (97751910 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


62/62 [==============================] - 361s 6s/step - loss: 1.9741 - acc: 0.3064 - val_loss: 1.9692 - val_acc: 0.3213
Epoch 2/10
62/62 [==============================] - 309s 5s/step - loss: 1.9052 - acc: 0.3472 - val_loss: 1.8977 - val_acc: 0.3350
Epoch 3/10
62/62 [==============================] - 310s 5s/step - loss: 1.8646 - acc: 0.3581 - val_loss: 1.8771 - val_acc: 0.3381
Epoch 4/10
62/62 [==============================] - 312s 5s/step - loss: 1.8450 - acc: 0.3635 - val_loss: 1.8947 - val_acc: 0.3528
Epoch 5/10
62/62 [==============================] - 307s 5s/step - loss: 1.8208 - acc: 0.3694 - val_loss: 1.8487 - val_acc: 0.3508
Epoch 6/10
62/62 [==============================] - 310s 5s/step - loss: 1.7862 - acc: 0.3805 - val_loss: 1.8424 - val_acc: 0.3589
Epoch 7/10
62/62 [==============================] - 304s 5s/step - loss: 1.7704 - acc: 0.3872 - val_loss: 1.8263 - val_acc: 0.3625
Epoch 8/10
62/62 [==============================] - 309s 5s/step - loss: 1.7512 - acc: 0.3987 

In [28]:
def save_model(model, train_batches):
    model_name = MODEL_PATH + f'carnet-{datetime.now().isoformat()}'

    model.save_weights(model_name + '-weights.h5')
    with open(model_name + '-model.json', 'w') as f:
        f.write(model.to_json())
    
    with open(model_name + '-classes.json', 'w') as f:
        json.dump(train_batches.class_indices, f)

In [15]:
from keras.models import model_from_json

In [18]:
def load_model():
    with open(MODEL_PATH + 'carnet-2017-11-12T00:04:15.433428-model.json') as f:
        model = model_from_json(f.read())
        
    model.load_weights(MODEL_PATH + 'carnet-2017-11-12T00:04:15.433428-weights.h5')
    
    return model

In [19]:
model = load_model()

In [25]:
model.optimizer.lr = 0.001

In [22]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=1,
)

Epoch 1/1


/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:888: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


122/123 [============================>.] - ETA: 4s - loss: 1.8275 - acc: 0.3887

/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (97751910 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


123/123 [==============================] - 651s 5s/step - loss: 1.8281 - acc: 0.3881 - val_loss: 1.7462 - val_acc: 0.4021


In [29]:
save_model(model, train_batches)

In [ ]:
for i in range(20):
    print(f'{i+1}/10')

    model.fit_generator(
        train_batches,
        steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
        validation_data=valid_batches,
        validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
        epochs=10,
    )
    
    save_model(model, train_batches)

1/10
Epoch 1/10


/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:888: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


122/123 [============================>.] - ETA: 4s - loss: 1.5167 - acc: 0.4864

/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (97751910 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


123/123 [==============================] - 655s 5s/step - loss: 1.5176 - acc: 0.4860 - val_loss: 1.6730 - val_acc: 0.4260
Epoch 2/10
123/123 [==============================] - 600s 5s/step - loss: 1.5358 - acc: 0.4820 - val_loss: 1.6253 - val_acc: 0.4581
Epoch 3/10
123/123 [==============================] - 600s 5s/step - loss: 1.5109 - acc: 0.4931 - val_loss: 1.5532 - val_acc: 0.4769
Epoch 4/10
123/123 [==============================] - 594s 5s/step - loss: 1.5251 - acc: 0.4898 - val_loss: 1.6144 - val_acc: 0.4453
Epoch 5/10
123/123 [==============================] - 598s 5s/step - loss: 1.5488 - acc: 0.4804 - val_loss: 1.5735 - val_acc: 0.4703
Epoch 6/10
123/123 [==============================] - 602s 5s/step - loss: 1.5371 - acc: 0.4837 - val_loss: 1.5996 - val_acc: 0.4601
Epoch 7/10
123/123 [==============================] - 598s 5s/step - loss: 1.5245 - acc: 0.4886 - val_loss: 1.6215 - val_acc: 0.4520
Epoch 8/10
123/123 [==============================] - 602s 5s/step - loss: 1.564